In [1]:
from elasticsearch import Elasticsearch
import urllib
import urllib.request
import json

#### Connections

In [2]:
username = 'els_g1ocfhb'
password = 'hqxn5zui4uf7po4r'
host = 'localhost'
port = 9220

In [3]:
es = Elasticsearch(
    ['http://{}:{}@{}:{}'.format(username, password, host, port)],
)

#### Basic info

In [4]:
health_status = es.cat.health(v=True)
print(health_status)

epoch      timestamp cluster      status node.total node.data shards pri relo init unassign pending_tasks max_task_wait_time active_shards_percent
1553724535 15:08:55  bds_ewlgx6ti yellow          1         1     11  11    0    0        1             0                  -                 91.7%



In [5]:
master_node = es.cat.master(v=True)
print(master_node)

id                     host        ip          node
PLAhZNNeTtONKb0ruoEMYg 10.44.11.99 10.44.11.99 DEV0003841.ESRI.COM



In [6]:
nodes_status = es.cat.nodes(v=True)
print(nodes_status)

ip          heap.percent ram.percent cpu load_1m load_5m load_15m node.role master name
10.44.11.99            7          40   7                          mdi       *      DEV0003841.ESRI.COM



In [7]:
es.cat.indices(v=True).split('\n')

['health status index                     uuid                   pri rep docs.count docs.deleted store.size pri.store.size',
 'yellow open   .dsconfig_es6-4-2         jfP1gSBSQxq_pJ93_fTWWQ   1   1          1            0     10.3kb         10.3kb',
 'green  open   esri-ds-metadata_es-6-4-2 -DAi720MS1qQc4Gm0Fl0vQ   5   0          0            0      1.2kb          1.2kb',
 'green  open   esri-ds-oid_es-6-4-2      VmsDszBeQDeEhV7Vlno2bg   5   0          0            0      1.2kb          1.2kb',
 '']

In [8]:
result = es.cat.indices(v=True).split('\n')
header = result[0]
health_index = header.split().index('health')
index_index = header.split().index('index')

In [9]:
index_list = list(map(lambda x: x.split()[index_index], result[1:-1]))
print(index_list)

['.dsconfig_es6-4-2', 'esri-ds-metadata_es-6-4-2', 'esri-ds-oid_es-6-4-2']


#### Read/Write

In [ ]:
mapping = {
    'properties': {
        'title': {
            'type': 'text',
            'analyzer': 'ik_max_word',
            'search_analyzer': 'ik_max_word'
        }
    }
}

In [10]:
es.indices.create(index='news', ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'news'}

In [11]:
result = es.indices.create(index='news', ignore=400)
print(result)

{'error': {'root_cause': [{'type': 'resource_already_exists_exception', 'reason': 'index [news/Stpw7GpsS-u_Mrq3pg-4ug] already exists', 'index_uuid': 'Stpw7GpsS-u_Mrq3pg-4ug', 'index': 'news'}], 'type': 'resource_already_exists_exception', 'reason': 'index [news/Stpw7GpsS-u_Mrq3pg-4ug] already exists', 'index_uuid': 'Stpw7GpsS-u_Mrq3pg-4ug', 'index': 'news'}, 'status': 400}


In [12]:
es.indices.delete(index='news', ignore=[400, 404])

{'acknowledged': True}

In [13]:
result = es.indices.create(index='movies_in_theaters', ignore=400)
print(result)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'movies_in_theaters'}


In [14]:
es.cat.indices(v=True).split('\n')

['health status index                     uuid                   pri rep docs.count docs.deleted store.size pri.store.size',
 'yellow open   .dsconfig_es6-4-2         jfP1gSBSQxq_pJ93_fTWWQ   1   1          1            0     10.3kb         10.3kb',
 'yellow open   movies_in_theaters        wF1f-MZnSMGKV8eqsGC4_Q   5   1          0            0      1.1kb          1.1kb',
 'green  open   esri-ds-metadata_es-6-4-2 -DAi720MS1qQc4Gm0Fl0vQ   5   0          0            0      1.2kb          1.2kb',
 'green  open   esri-ds-oid_es-6-4-2      VmsDszBeQDeEhV7Vlno2bg   5   0          0            0      1.2kb          1.2kb',
 '']

In [15]:
url = 'https://api.douban.com/v2/movie/in_theaters?apikey=0b2bdeda43b5688921839c8ecb20399b&city=%E5%8C%97%E4%BA%AC&start=0&count=100&client=&udid='
request = urllib.request.Request(url)
response = urllib.request.urlopen(request).read().decode('utf-8')
content = json.loads(response)
movies = content['subjects']
movies_data = []
for item in movies:
    data = {}
    data['title'] = item['title']
    data['original_title'] = item['original_title']
    data['rating'] = item['rating']['average']
    data['genres'] = item['genres']
    data['pubdates'] = item['pubdates']
    movies_data.append(data)
    es.index(index='movies_in_theaters', doc_type='info', body=data)

In [16]:
es.cat.indices(v=True).split('\n')

['health status index                     uuid                   pri rep docs.count docs.deleted store.size pri.store.size',
 'yellow open   .dsconfig_es6-4-2         jfP1gSBSQxq_pJ93_fTWWQ   1   1          1            0     10.3kb         10.3kb',
 'yellow open   movies_in_theaters        wF1f-MZnSMGKV8eqsGC4_Q   5   1         30            0       41kb           41kb',
 'green  open   esri-ds-metadata_es-6-4-2 -DAi720MS1qQc4Gm0Fl0vQ   5   0          0            0      1.2kb          1.2kb',
 'green  open   esri-ds-oid_es-6-4-2      VmsDszBeQDeEhV7Vlno2bg   5   0          0            0      1.2kb          1.2kb',
 '']

In [ ]:
result = es.search(index='movies_in_theaters', doc_type='info', ignore=404)
print(json.dumps(result, indent=2, ensure_ascii=False))

In [ ]:
query = {
    'query': {
        'bool': {
            'must': [
                { 'match': { 'genres': '剧情' }},
                { 'match': { 'pubdates': { 'query': '2018 2019 美国 中国大陆', 'operator': 'and' }}}
            ]
        }
    }
}
result = es.search(index='movies_in_theaters', doc_type='info', body=query)
print(json.dumps(result, indent=2, ensure_ascii=False))

In [19]:
result = es.indices.delete(index='movies_in_theaters', ignore=400)
print(result)

{'acknowledged': True}


In [20]:
es.cat.indices(v=True).split('\n')

['health status index                     uuid                   pri rep docs.count docs.deleted store.size pri.store.size',
 'yellow open   .dsconfig_es6-4-2         jfP1gSBSQxq_pJ93_fTWWQ   1   1          1            0     10.3kb         10.3kb',
 'green  open   esri-ds-metadata_es-6-4-2 -DAi720MS1qQc4Gm0Fl0vQ   5   0          0            0      1.2kb          1.2kb',
 'green  open   esri-ds-oid_es-6-4-2      VmsDszBeQDeEhV7Vlno2bg   5   0          0            0      1.2kb          1.2kb',
 '']